In [ ]:
%%bash
wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
tar -xf aclImdb_v1.tar.gz

In [1]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir == "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [2]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [3]:
import pandas as pd

train = pd.DataFrame({'text': train_texts, 'label': train_labels})
val = pd.DataFrame({'text': val_texts, 'label': val_labels})
test = pd.DataFrame({'text': test_texts, 'label': test_labels})

In [22]:
import os

os.makedirs('data/train', exist_ok=True)
os.makedirs('data/val', exist_ok=True)
os.makedirs('data/test', exist_ok=True)

train.to_csv('data/train.csv', index=False)
val.to_csv('data/val.csv', index=False)
test.sample(200).to_csv('data/test.csv', index=False)

In [6]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/imdb"

role = sagemaker.get_execution_role()

In [23]:
inputs_train = sagemaker_session.upload_data("data/train.csv", bucket=bucket, key_prefix=prefix)
inputs_test = sagemaker_session.upload_data("data/test.csv", bucket=bucket, key_prefix=prefix)
inputs_val = sagemaker_session.upload_data("data/val.csv", bucket=bucket, key_prefix=prefix)

In [16]:
inputs_train

's3://sagemaker-eu-central-1-611215368770/sagemaker/imdb/train.csv'

In [10]:
import pandas as pd
import io
import boto3

In [11]:
s3_resource = boto3.resource('s3')

In [25]:
bucket = "sagemaker-eu-central-1-611215368770"
response = s3_resource.Bucket(bucket).Object(key="sagemaker/imdb/test.csv").get()
data = pd.read_csv(io.BytesIO(response["Body"].read()), encoding='latin1')
data

,text,label
0,"b""This is what movies should aspire to. Funny ...",1
1,"b""'Thriller' remains the greatest of the pop m...",1
2,b'Hi I have been looking 4 the soundtrack or a...,1
3,"b""I just spent about 1.5 hours waiting for the...",0
4,"b""When you watch the making of with this DVD -...",0
...,...,...
195,b'Calling this film a decent or enjoyable horr...,0
196,"b""Is Thursday an original film? Heck no, but i...",1
197,b'This was one of the shows that I wanted to f...,0
198,"b'Before I had seen this film, I had heard som...",0
